In [ ]:
!pip install jupysql

%load_ext sql
%sql mysql+pymysql://app:app_password@db/sakila_star

<a id='ex01'></a>
### Exercise 1

Write an SQL query to get the total amount obtained from the renting of Travel, Family, and Children films during the months of June and July of 2005. Present the results grouped and ordered by store id and category name.

*Note*: Agood practice when exploring your tables for the first time is to set a `LIMIT`, mostly if you want to get all columns from a table with the `*` wildcard.

In [14]:
%%sql
SELECT
    fact_rental.store_id,
    dim_category.name AS category_name,
    SUM(fact_rental.amount) AS total_amount
FROM 
    dim_category
    INNER JOIN fact_rental ON fact_rental.category_id = dim_category.category_id
WHERE 
    fact_rental.rental_date BETWEEN "2005-06-01" AND "2005-08-01"
    AND dim_category.name IN ("Travel", "Family", "Children")
GROUP BY 
    fact_rental.store_id, 
    dim_category.name
ORDER BY 
    fact_rental.store_id, 
    dim_category.name 

Running query in 'mysql+pymysql://app:***@db/sakila_star'

6 rows affected.

store_id,category_name,total_amount
1,Children,1013.39
1,Family,1249.83
1,Travel,928.78
2,Children,1022.25
2,Family,1168.03
2,Travel,1078.53


In [15]:
%%sql
WITH actor_film AS (
    SELECT DISTINCT 
        dim_actor.actor_id,
        dim_actor.first_name,
        dim_actor.last_name,
        dim_film.film_id
    FROM
        dim_actor
        INNER JOIN bridge_actor ON dim_actor.actor_id = bridge_actor.actor_id
        INNER JOIN fact_rental ON bridge_actor.rental_id = fact_rental.rental_id
        INNER JOIN dim_film ON fact_rental.film_id = dim_film.film_id
),
actor_film_count AS (
    SELECT
        actor_id,
        first_name,
        last_name,
        COUNT(*) AS films
    FROM
        actor_film
    GROUP BY
        actor_id
)
SELECT
    first_name,
    last_name,
    films,
    ROW_NUMBER() over (
        ORDER BY
            films DESC, first_name, last_name ASC
    ) AS actor_rank
FROM
    actor_film_count
LIMIT 10;

Running query in 'mysql+pymysql://app:***@db/sakila_star'

10 rows affected.

first_name,last_name,films,actor_rank
GINA,DEGENERES,42,1
MARY,KEITEL,40,2
WALTER,TORN,39,3
MATTHEW,CARREY,37,4
SANDRA,KILMER,36,5
ANGELA,WITHERSPOON,35,6
HENRY,BERRY,35,7
SCARLETT,DAMON,35,8
VAL,BOLGER,35,9
VIVIEN,BASINGER,35,10
